In [1]:
import tensorflow_datasets as tfds  # TFDS to download MNIST.
import tensorflow as tf  # TensorFlow / `tf.data` operations.
from flax import nnx  # The Flax NNX API.
from functools import partial
import jax.numpy as jnp  # JAX NumPy
import jax
import optax
from linearRNN import forward_h
from linearRNN import forward
from linearRNN import init_lru_parameters
import numpy as np
from flax import linen as nn


In [ ]:
rnn=1 #rnn=0:transformation of the inputs with fixed RNN weights, rnn=1: adding the RNN module on the model to learn the weight matrices
hidden_neuron=256
hidden_size=2048
learning_rate = 0.005
momentum = 0.9
train_steps=2500
eval_every = 100
batch_size=32
method_name="LRUMLP"
dataset_name="MNIST"

In [3]:
#Import data
cifar=tf.keras.datasets.mnist.load_data()
train=cifar[0]
test=cifar[1]

train_x_len=train[0].shape[0]
train_x_size=int(jnp.prod(jnp.array(train[0].shape[1:])))
test_x_len=test[0].shape[0]
test_x_size=int(jnp.prod(jnp.array(test[0].shape[1:])))

train_x=train[0].reshape((train_x_len,train_x_size))/255
train_y=train[1].reshape(train_x_len)


test_x=test[0].reshape((test_x_len,test_x_size))/255
test_y=test[1].reshape(test_x_len)

#test_x=jnp.append(train_x[30000:],test_x,axis=0)
#test_y=jnp.append(train_y[30000:],test_y)
#
#train_x=train_x[:30000]
#train_y=train_y[:30000]

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [4]:
if rnn:
    train_hidden_state=train_x
    test_hidden_state=test_x
else:
    param=init_lru_parameters(hidden_size,train_x_size,r_min=0.999,r_max=0.9999,max_phase=6.28)
    param2=init_lru_parameters(hidden_size,train_x_size,r_min=0.999,r_max=0.9999,max_phase=6.28)
    train_hidden_state=jnp.real(forward_h(param,train_x))
    test_hidden_state=jnp.real(forward_h(param2,test_x))

In [5]:
train_ds=tf.data.Dataset.from_tensor_slices((jnp.real(train_hidden_state),jnp.array(train_y,dtype=int)))
test_ds=tf.data.Dataset.from_tensor_slices((jnp.real(test_hidden_state),jnp.array(test_y,dtype=int)))

train_ds = train_ds.repeat().shuffle(100)
# Group into batches of `batch_size` and skip incomplete batches, prefetch the next sample to improve latency.
train_ds = train_ds.batch(batch_size, drop_remainder=True).take(train_steps).prefetch(1)
# Group into batches of `batch_size` and skip incomplete batches, prefetch the next sample to improve latency.
test_ds = test_ds.batch(batch_size, drop_remainder=True).prefetch(1)

In [6]:
print(train_hidden_state.shape)
print(test_hidden_state.shape)

(60000, 784)
(10000, 784)


In [ ]:
from flax import nnx
import optax
from flax.nnx.nn.recurrent import LSTMCell,GRUCell
from func import LRNNCell,LRUCell

class MLP(nnx.Module):
  def __init__(self, din, dmid, dout, rngs: nnx.Rngs):
    self.rnn=nnx.RNN(LRUCell(in_features=train_x_size, hidden_features=din, rngs=nnx.Rngs(0)))
    self.linear = nnx.Linear(din, dmid, rngs=rngs)
    self.linear_out = nnx.Linear(dmid, dout, rngs=rngs)

  def __call__(self, x):
    if rnn:
      x=self.rnn(x)
    x=self.linear(x)
    x = nnx.softplus(x)
    return self.linear_out(x)

model = MLP(hidden_size, hidden_neuron, 10, rngs=nnx.Rngs(0))  # eager initialization

nnx.display(model)

MLP(
  rnn=RNN(
    cell=LSTMCell(
      in_features=784,
      hidden_features=2048,
      gate_fn=<PjitFunction of <function sigmoid at 0x00000207EFD39080>>,
      activation_fn=<PjitFunction of <function tanh at 0x00000207EEA91580>>,
      kernel_init=<function variance_scaling.<locals>.init at 0x00000207F2BD2F20>,
      recurrent_kernel_init=<function modified_orthogonal at 0x00000207F2C05120>,
      bias_init=<function zeros at 0x00000207EFC658A0>,
      dtype=None,
      param_dtype=<class 'jax.numpy.float32'>,
      carry_init=<function zeros at 0x00000207EFC658A0>,
      rngs=Rngs(
        default=RngStream(
          key=RngKey(
            value=Array((), dtype=key<fry>) overlaying:
            [0 0],
            tag='default'
          ),
          count=RngCount(
            value=Array(12, dtype=uint32),
            tag='default'
          )
        )
      ),
      ii=Linear(
        kernel=Param(
          value=Array(shape=(784, 2048), dtype=float32)
        ),
        

In [8]:
import optax

optimizer = nnx.Optimizer(model, optax.adamw(learning_rate, momentum))
metrics = nnx.MultiMetric(
  accuracy=nnx.metrics.Accuracy(),
  loss=nnx.metrics.Average('loss'),
)

nnx.display(optimizer)

Optimizer(
  step=OptState(
    value=Array(0, dtype=uint32)
  ),
  model=MLP(
    rnn=RNN(
      cell=LSTMCell(
        in_features=784,
        hidden_features=2048,
        gate_fn=<PjitFunction of <function sigmoid at 0x00000207EFD39080>>,
        activation_fn=<PjitFunction of <function tanh at 0x00000207EEA91580>>,
        kernel_init=<function variance_scaling.<locals>.init at 0x00000207F2BD2F20>,
        recurrent_kernel_init=<function modified_orthogonal at 0x00000207F2C05120>,
        bias_init=<function zeros at 0x00000207EFC658A0>,
        dtype=None,
        param_dtype=<class 'jax.numpy.float32'>,
        carry_init=<function zeros at 0x00000207EFC658A0>,
        rngs=Rngs(
          default=RngStream(
            key=RngKey(
              value=Array((), dtype=key<fry>) overlaying:
              [0 0],
              tag='default'
            ),
            count=RngCount(
              value=Array(12, dtype=uint32),
              tag='default'
            )
          )
 

In [9]:
import jax
def loss_fn(model: MLP, batch):
  logits = model(batch[0])
  loss = optax.softmax_cross_entropy_with_integer_labels(
    logits=logits, labels=batch[1]
  ).mean()
  #print(logits.shape)
  #print(batch[1].shape)
  return loss, logits

@nnx.jit
def train_step(model: MLP, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch):
  """Train for a single step."""
  grad_fn = nnx.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(model, batch)
  metrics.update(loss=loss, logits=logits, labels=batch[1])  # In-place updates.
  optimizer.update(grads)  # In-place updates.
  predicted_labels = jnp.argmax(logits, axis=-1)
  actual_labels = batch[1]
  #jax.debug.print("Predictions: {}",predicted_labels[:5].astype(int))
  #jax.debug.print("Actual Labels: {}",actual_labels[:5].astype(int))
  
@nnx.jit
def eval_step(model: MLP, metrics: nnx.MultiMetric, batch):
  loss, logits = loss_fn(model, batch)
  metrics.update(loss=loss, logits=logits, labels=batch[1])  # In-place updates.

In [ ]:
#Train the model + evaluation with the test data
metrics_history = {
    'train_loss': [],
    'train_accuracy': [],
    'test_loss': [],
    'test_accuracy': [],
}

for step, batch in enumerate(train_ds.as_numpy_iterator()):
  # Run the optimization for one step and make a stateful update to the following:
  # - The train state's model parameters
  # - The optimizer state
  # - The training loss and accuracy batch metrics
  train_step(model, optimizer, metrics, batch)

  if step > 0 and (step % eval_every == 0 or step == train_steps - 1):  # One training epoch has passed.
    # Log the training metrics.
    for metric, value in metrics.compute().items():  # Compute the metrics.
      metrics_history[f'train_{metric}'].append(value)  # Record the metrics.
    metrics.reset()  # Reset the metrics for the test set.

    # Compute the metrics on the test set after each training epoch.
    for test_batch in test_ds.as_numpy_iterator():
      eval_step(model, metrics, test_batch)

    # Log the test metrics.
    for metric, value in metrics.compute().items():
      metrics_history[f'test_{metric}'].append(value)
    metrics.reset()  # Reset the metrics for the next training epoch.

    print(
      f"[train] step: {step}, "
      f"loss: {metrics_history['train_loss'][-1]}, "
      f"accuracy: {metrics_history['train_accuracy'][-1] * 100}"
    )
    print(
      f"[test] step: {step}, "
      f"loss: {metrics_history['test_loss'][-1]}, "
      f"accuracy: {metrics_history['test_accuracy'][-1] * 100}"
    )

[train] step: 100, loss: 1.1737847328186035, accuracy: 66.58415985107422
[test] step: 100, loss: 0.4096709191799164, accuracy: 87.23957824707031
[train] step: 200, loss: 0.36811864376068115, accuracy: 88.875
[test] step: 200, loss: 0.3984302580356598, accuracy: 87.67027282714844
[train] step: 300, loss: 0.3494749069213867, accuracy: 89.03125
[test] step: 300, loss: 0.32557395100593567, accuracy: 89.71354675292969
[train] step: 400, loss: 0.2804947793483734, accuracy: 91.90625
[test] step: 400, loss: 0.32973578572273254, accuracy: 88.80207824707031
[train] step: 500, loss: 0.27946534752845764, accuracy: 91.125
[test] step: 500, loss: 0.21903975307941437, accuracy: 93.03886413574219
[train] step: 600, loss: 0.20861873030662537, accuracy: 93.5625
[test] step: 600, loss: 0.20950168371200562, accuracy: 93.359375


In [ ]:
#Save the training results into csv
import pandas as pd


data=pd.DataFrame({"step":np.arange(eval_every,train_steps+eval_every,eval_every),"train_loss":metrics_history['train_loss'],
                   "test_loss":metrics_history['test_loss'],"train_accuracy":metrics_history['train_accuracy'],
                   "test_accuracy":metrics_history['test_accuracy']})
data.to_csv(method_name+"_nr"+str(hidden_neuron)+"_d"+str(hidden_size)+"_"+dataset_name+"_step"+str(train_steps)+".csv")

In [ ]:
#Plot the loss
import matplotlib.pyplot as plt

plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['train_loss'],label="train loss")
plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['test_loss'],label="test loss")
plt.title("Train loss of MNIST dataset with GRU+MLP, \nhidden dimension="+str(hidden_size)+", number of neuron="+str(hidden_neuron))
plt.xlabel("Training step")
plt.ylabel("Train loss (cross entropy)")
plt.legend()
plt.savefig("loss_"+method_name+"_"+str(hidden_neuron)+"_"+dataset_name+"_step"+str(train_steps)+".jpg")
plt.show()

In [ ]:
#Plot the accuracy
plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['train_accuracy'],label="train")
plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['test_accuracy'],label="test")
plt.title("Accuracy of MNIST dataset with LRUfix+MLP, \nhidden dimension="+str(hidden_size)+", number of neuron="+str(hidden_neuron))
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("accuracy_"+method_name+"_"+str(hidden_neuron)+"_"+dataset_name+"_step"+str(train_steps)+".jpg")
plt.show()